# Failure Analysis

## Imports

### Packages

In [1]:
import torch
import torch.nn as nn
from torch.utils import data
from torchvision import models
import pandas as pd
import numpy as np
import argparse
import sys
import os
import argparse
from tqdm import tqdm

### Local

In [2]:
sys.path.append(".")
sys.path.append("..")
from dataset.patient_dataset import PatientDataset
from models.lstm import LSTM
from models.omnipotent_resnet import Net
from models.combined_net import CombinedNet

## Parameters

In [3]:
### DEFAULT PARAMETERS ###
### Data parameters ###
DATA_DIR = '../../../data/sliced-data/train'  # The slices we will train on for each patient
DS_DIR = '../../../data/data-split'
TARGET_SLICES = (0, 32)
### Model parameters ###
MODEL_DIR = '../../../models'  # Directory where best models are saved
TRAINED_MODEL = 'lstm_008.pt'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Train on GPU or CPU
RESNET_MODEL_TYPE = 'resnet34'  # Which type of resnet is used by the model
### Test parameters ###
N_FEATURES = 128
BATCH_SIZE = 1

## Load Model

In [4]:
model = models.resnet34()
resnet = Net(model, 'combinednet', N_FEATURES)
lstm_net = LSTM(n_features=N_FEATURES, n_hidden=64, n_layers=2)
combined_net = CombinedNet(name='combinednet', cnn_net=resnet, lstm_net=lstm_net)
combined_net.load_state_dict(torch.load(os.path.join(MODEL_DIR, TRAINED_MODEL)))
combined_net.to(DEVICE)

CombinedNet(
  (cnn_net): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-0

## Load Data

In [5]:
val_df = pd.read_csv(os.path.join(DS_DIR, "val_df.csv"), names=["patient_nr", "slice_nr", "class"])
print(f"\nNumber of unique patient numbers in validation set: {len(np.unique(val_df['patient_nr']))}")
print(f"Number of unique slice numbers in validation set:   {len(np.unique(val_df['slice_nr']))}")
print(f"Number of unique class values in validation set:    {len(np.unique(val_df['class']))}")
    
val_patients = pd.read_csv(os.path.join(DS_DIR, "val_patients.csv"), names=["patient_nr"]).to_numpy().flatten()
print(f"Number of patient numbers in the validation patients list: {len(val_patients)}")


Number of unique patient numbers in validation set: 2
Number of unique slice numbers in validation set:   32
Number of unique class values in validation set:    2
Number of patient numbers in the validation patients list: 2


In [6]:
print(val_df)

     patient_nr  slice_nr  class
160         168         0   True
161         168         1   True
162         168         2   True
163         168         3   True
164         168         4   True
..          ...       ...    ...
347         958        27  False
348         958        28  False
349         958        29  False
350         958        30  False
351         958        31  False

[64 rows x 3 columns]


##  Dataloader

In [7]:
analysis_set = PatientDataset(val_df, val_patients, TARGET_SLICES, DATA_DIR)
analysis_loader = data.DataLoader(analysis_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=os.cpu_count())

## Test Model

In [8]:
combined_net.eval()
false_positives = []
false_negatives = []
    
for batch_idx, (images, targets) in tqdm(enumerate(analysis_loader), total=len(analysis_loader), desc="#test_batches", leave=False):
    images = images.float().to(DEVICE)
    target = targets.int().detach().cpu().data.numpy()[0]
    output = combined_net(images).detach().cpu()
    images = images.detach().cpu().data.numpy()
    
    # Compute probabilities (requirement: round to 5 decimals)
    probabilities = np.round(torch.sigmoid(output).numpy(), 5)
    
    # Compute class from probability (>0.5 = abnormal)
    klass = (probabilities > 0.5).astype(np.uint8)
    
    print('target: '+str(target))
    print('class: '+str(klass))
    
    if((target == 0) & (klass == 1)):
        false_positives.append(images)
        
    if((target == 1) & (klass == 0)):
        false_negatives.append(images)

#test_batches:  50%|█████     | 1/2 [00:01<00:01,  1.32s/it]

target: 0
class: 0


target: 1
class: 1


In [9]:
false_positives

[]

In [10]:
false_negatives

[]